In [454]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
from gensim import corpora, models
from sklearn import preprocessing
%matplotlib inline

In [456]:
database = pd.read_csv('sampleEHR2.csv')

In [458]:
database

,age,sex,cholesterol,triglyceride,HDL,LDL,PathDiagNum,BMI,ALT,AST,glucose,cirrhosis
0,63,1,103,147,35,38.6,1,20.0,27,35,117,1
1,68,2,141,95,38,84.0,1,23.1,78,74,98,0
2,79,1,143,71,60,68.8,1,21.3,44,40,95,1
3,52,1,126,64,39,74.2,1,34.0,18,26,101,1
4,77,2,126,49,41,75.2,1,25.7,106,97,128,1
...,...,...,...,...,...,...,...,...,...,...,...,...
117,64,2,205,57,68,125.6,1,19.9,38,41,109,0
118,65,2,143,42,53,81.6,1,27.8,74,75,136,0
119,66,1,223,89,47,158.2,1,24.3,43,39,120,0
120,64,1,152,82,56,79.6,1,18.0,35,50,91,1


In [460]:
connection = sql.connect('my_data2.db')

In [462]:
database.to_sql('peoples', connection, if_exists='replace', index=False)

122

In [464]:
query = '''
SELECT age, sex, cholesterol, triglyceride, HDL, LDL, PathDiagNum, BMI, ALT, AST, glucose, cirrhosis 
FROM peoples
LIMIT 122
'''

In [466]:
results = pd.read_sql(query, connection)
print(results)

     age  sex  cholesterol  triglyceride  HDL    LDL  PathDiagNum   BMI  ALT  \
0     63    1          103           147   35   38.6            1  20.0   27   
1     68    2          141            95   38   84.0            1  23.1   78   
2     79    1          143            71   60   68.8            1  21.3   44   
3     52    1          126            64   39   74.2            1  34.0   18   
4     77    2          126            49   41   75.2            1  25.7  106   
..   ...  ...          ...           ...  ...    ...          ...   ...  ...   
117   64    2          205            57   68  125.6            1  19.9   38   
118   65    2          143            42   53   81.6            1  27.8   74   
119   66    1          223            89   47  158.2            1  24.3   43   
120   64    1          152            82   56   79.6            1  18.0   35   
121   68    2          152           139   76   48.2            1  26.9   84   

     AST  glucose  cirrhosis  
0     35

In [468]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA  # Import PCA from sklearn

In [470]:
X = results[['age', 'sex', 'cholesterol', 'triglyceride','HDL', 'LDL', 'PathDiagNum','BMI', 'ALT', 'AST', 'glucose']]
y = results['cirrhosis']

In [472]:
if X.isnull().values.any():
    raise ValueError("Missing values found in the data.")

In [474]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [476]:
pca = PCA(n_components=2)  # Specify the number of principal components
X_pca = pca.fit_transform(X_scaled)

In [478]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

In [480]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)

# Predict on the test data
y_pred = knn.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Example prediction
example_data = [[80, 1, 168, 190, 70, 60, 1, 20.8, 74, 77, 134]]  # Replace with actual values
example_data_scaled = scaler.transform(example_data)
example_data_pca = pca.transform(example_data_scaled)
example_prediction = knn.predict(example_data_pca)
print("Predicted customer category:", example_prediction)

Accuracy: 0.4594594594594595
              precision    recall  f1-score   support

           0       0.32      0.73      0.44        11
           1       0.75      0.35      0.47        26

    accuracy                           0.46        37
   macro avg       0.54      0.54      0.46        37
weighted avg       0.62      0.46      0.46        37

Predicted customer category: [1]


C:\Users\ki2282\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
